In [51]:
import baserun
import csv
import openai
import os
import tiktoken
import json
from pydantic import BaseModel
from tqdm.notebook import tqdm
from typing import Tuple
from tenacity import retry, wait_random_exponential

from dotenv import load_dotenv

Set API keys

In [30]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
baserun.init()

/Users/rushabh/Library/Caches/pypoetry/virtualenvs/gptsaregpts-bRn197-3-py3.11/lib/python3.11/site-packages/baserun/baserun.py:44: UserWarning: Baserun has already been initialized. Additional calls to init will be ignored.
  warnings.warn("Baserun has already been initialized. Additional calls to init will be ignored.")


Connect to the database

In [31]:
import sqlite3


def connect(name: str) -> Tuple[sqlite3.Connection, sqlite3.Cursor]:
    conn = sqlite3.connect(name)
    c = conn.cursor()
    return conn, c

In [32]:
conn, c = connect("../db.sqlite")

Get the tokens for the classifier.

In [33]:
def get_tokens() -> dict:
    encoding = tiktoken.encoding_for_model("gpt-4")
    tokens = [encoding.encode(label) for label in ["0", "1", "2", "3", "N"]]

    # Flatten the list of lists and convert to a set.
    token_set = set([item for sublist in tokens for item in sublist])

    # Create a dictionary with the tokens and a default value of 100.
    token_dict = {str(token): 100 for token in token_set}

    return token_dict


logit_bias = get_tokens()
print(json.dumps(logit_bias))

{"45": 100, "15": 100, "16": 100, "17": 100, "18": 100}


~~Create the classification table if it doesn't exist already.~~

We don't need a classification table anymore.

In [34]:
def create_classification_table(c: sqlite3.Cursor, conn: sqlite3.Connection) -> None:
    dwa_classification_sql = """
    CREATE TABLE IF NOT EXISTS dwa_classification (
        onetsoc_code CHARACTER(10) NOT NULL,
        task_id DECIMAL(8,0) NOT NULL,    
        dwa_id CHARACTER VARYING(20) NOT NULL,
        classification CHARACTER(2) NOT NULL,
        FOREIGN KEY (onetsoc_code) REFERENCES occupation_data(onetsoc_code),
        FOREIGN KEY (task_id) REFERENCES task_statements(task_id),
        FOREIGN KEY (dwa_id) REFERENCES dwa_reference(dwa_id),
        PRIMARY KEY (onetsoc_code, task_id, dwa_id)
    );
    """

    c.execute(dwa_classification_sql)
    conn.commit()


# create_classification_table(c, conn)

Setup the prompts.

In [35]:
from pathlib import Path

user_template = Path(Path.cwd().parent / "prompts" / "user.txt").read_text()
system_template = Path(Path.cwd().parent / "prompts" / "system.txt").read_text()

Fetch all the rows from the database.

In [36]:
class DWAReference(BaseModel):
    dwa_id: str
    dwa_title: str
    classification: str | None = None

    @classmethod
    def from_tuple(cls, tup: tuple):
        return cls(
            dwa_id=tup[0],
            dwa_title=tup[1],
        )
    
    @retry(wait=wait_random_exponential(multiplier=1, max=60))
    async def classify(self) -> str:
        user_message = {
            "role": "user",
            "content": user_template.format(
                task=self.dwa_title,
            ),
        }
        system_message = {"role": "system", "content": system_template}

        response = await openai.ChatCompletion.acreate(
            messages=[system_message, user_message],
            logit_bias=logit_bias,
            max_tokens=1,
            temperature=0,
            model="gpt-3.5-turbo",
        )
        self.classification = response.choices[0]["message"]["content"]
        return self.classification

    def save(self, c: sqlite3.Cursor) -> None:
        insert_sql = "UPDATE dwa_reference SET classification = ? WHERE dwa_id = ?"
        c.execute(insert_sql, (self.classification, self.dwa_id))

In [37]:
def fetch_rows(c: sqlite3.Cursor, num: int = -1) -> list:
    select_query = """
select    
  t.dwa_id,
  t.dwa_title
from
  dwa_reference as t
where
  t.classification is null
order by
  t.dwa_id
  """
    c.execute(select_query)
    if num == -1:
        return [DWAReference.from_tuple(r) for r in c.fetchall()]
    else:
        return [DWAReference.from_tuple(r) for r in c.fetchmany(num)]

In [38]:
async def classify_all(rows: list[DWAReference]) -> None:
    for row in tqdm(rows, desc="Classifying"):
        await row.classify()
        print(f"[{row.dwa_id}]: {row.dwa_title}: {row.classification}")
        row.save(c)
    conn.commit()

In [42]:
occupation_codes = fetch_rows(c, 1000)
await classify_all(occupation_codes)

Classifying: 0it [00:00, ?it/s]

In [45]:
def calculate_task_scores():
    calculation_sql = """
    UPDATE task_statements
    SET 
        alpha = (
            SELECT 
                1.0 * SUM(CASE WHEN classification = 1 THEN 1 ELSE 0 END) / COUNT(*)
            FROM 
                tasks_to_dwas
            JOIN 
                dwa_reference ON tasks_to_dwas.dwa_id = dwa_reference.dwa_id
            WHERE
                tasks_to_dwas.task_id = task_statements.task_id
        ),
        beta = (
            SELECT 
                1.0 * (SUM(CASE WHEN classification = 1 THEN 1 ELSE 0 END) + 0.5 * SUM(CASE WHEN classification = 2 THEN 1 ELSE 0 END)) / COUNT(*)
            FROM 
                tasks_to_dwas
            JOIN 
                dwa_reference ON tasks_to_dwas.dwa_id = dwa_reference.dwa_id
            WHERE
                tasks_to_dwas.task_id = task_statements.task_id
        ),
        zeta = (
            SELECT 
                1.0 * (SUM(CASE WHEN classification = 1 THEN 1 ELSE 0 END) + SUM(CASE WHEN classification = 2 THEN 1 ELSE 0 END) + SUM(CASE WHEN classification = 3 THEN 1 ELSE 0 END)) / COUNT(*)
            FROM 
                tasks_to_dwas
            JOIN 
                dwa_reference ON tasks_to_dwas.dwa_id = dwa_reference.dwa_id
            WHERE
                tasks_to_dwas.task_id = task_statements.task_id
        )
    WHERE task_id IN (SELECT task_id FROM tasks_to_dwas);
    """
    c.execute(calculation_sql);
    conn.commit();

# calculate_task_scores()

In [47]:
def calculate_occupation_scores():
    occupation_data_sql = """
        UPDATE occupation_data
        SET
            alpha = (
                SELECT AVG(t.alpha)
                FROM task_statements as t
                WHERE t.onetsoc_code = occupation_data.onetsoc_code
            ),
            beta = (
                SELECT AVG(t.beta)
                FROM task_statements as t
                WHERE t.onetsoc_code = occupation_data.onetsoc_code
            ),
            zeta = (
                SELECT AVG(t.zeta)
                FROM task_statements as t
                WHERE t.onetsoc_code = occupation_data.onetsoc_code
            )
        ;
    """
    c.execute(occupation_data_sql);
    conn.commit();

# calculate_occupation_scores()

## Notes

Industry and employment numbers from BLS
https://www.bls.gov/emp/data/industry-out-and-emp.htm

CPS ONET crosswalks
https://www.bls.gov/emp/documentation/crosswalks.htm

We have to somehow map the major industry group by alpha, beta, and zeta. Steps:
1. Major industry group NAICS code -> List of NEM codes.
2. ✅ NEM codes -> ONET codes.
3. Calculate scores per major industry group.

Breakthrough!
- EP data tables: https://www.bls.gov/emp/tables.htm
- [industry-occupation-matrix by industry](https://www.bls.gov/emp/tables/industry-occupation-matrix-industry.htm)
- [industry-occupation-matrix by occupation](https://www.bls.gov/emp/tables/industry-occupation-matrix-occupation.htm)

There is some annoyance in having to download the individual CSV files, but we can get over that. This should help us do weighted exposure per industry. 
Then we can back into major industries by output and by employment.


In [52]:
def populate_onet_to_nem_mapping(csv_filepath: Path):
    # Open and read the CSV file
    with open(csv_filepath, 'r') as f:
        csv_reader = csv.DictReader(f)
        
        # Loop through each row in the CSV file
        for row in csv_reader:
            onetsoc_code = row['O*NET-SOC Code']
            nem_code = row['NEM Code']
            nem_title = row['National Employment Matrix Occupational Title']
            ooh_profile_code = row['OOH Profile Code']
            ooh_profile_title = row['OOH Profile Title']
            ooh_occupation_group = row['OOH Occupation Group']
            ooh_profile_website = row['OOH Profile Website']

            # Insert data into the SQLite table
            c.execute("""
                INSERT INTO bls_onet
                (onetsoc_code, nem_code, nem_title, ooh_profile_code, ooh_profile_title, ooh_occupation_group, ooh_profile_website)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (onetsoc_code, nem_code, nem_title, ooh_profile_code, ooh_profile_title, ooh_occupation_group, ooh_profile_website))
    
    # Commit changes and close the connection
    conn.commit()
    print("Added rows to bls_onet table.")

# Usage
# populate_onet_to_nem_mapping(Path.cwd().parent / "data" / "bls" / "onet_soc_crosswalk.csv")


In [53]:
def load_bls_matrix_csv_to_db(csv_file_path: Path):
    with open(csv_file_path, newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)  # Skip header row

        for row in tqdm(csvreader):
            occupation_type = row[0]
            industry_type = row[1]
            occupation_code = row[2]
            occupation_title = row[3]
            industry_code = row[4]
            industry_title = row[5]
            employment_2022 = float(row[6])
            percent_industry_2022 = float(row[7])
            percent_occupation_2022 = float(row[8])

            c.execute("""
                INSERT INTO bls_matrix (
                    occupation_type,
                    industry_type,
                    occupation_code,
                    occupation_title,
                    industry_code,
                    industry_title,
                    employment_2022,
                    percent_industry_2022,
                    percent_occupation_2022
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (occupation_type, industry_type, occupation_code, occupation_title, industry_code, industry_title, employment_2022, percent_industry_2022, percent_occupation_2022))

    conn.commit()

# Use the function
# load_bls_matrix_csv_to_db(Path.cwd().parent / "data" / "bls" / "matrix.csv")


In [68]:
def calculate_bls_occupation_scores():
    # Fetch all rows where the occupation type is "Line item" and the industry type is "Line item"
    fetch_rows_sql = """
SELECT
    bm.id,
    bm.alpha,
    bm.beta,
    bm.zeta,
    od.alpha,
    od.beta,
    od.zeta
FROM
    occupation_data AS od
JOIN
    bls_onet AS bo ON od.onetsoc_code = bo.onetsoc_code
JOIN
    bls_matrix as bm ON bm.occupation_code = bo.nem_code
WHERE
    bm.occupation_type = 'Line item' AND
    bm.industry_type = 'Line item' AND
    bm.alpha IS NULL AND
    od.alpha IS NOT NULL
"""
    for rows in tqdm(c.execute(fetch_rows_sql).fetchall()):
        id, alpha, beta, zeta, od_alpha, od_beta, od_zeta = rows
        update_row_sql = """
UPDATE bls_matrix
SET
    alpha = ?,
    beta = ?,
    zeta = ?
WHERE
    id = ?
"""
        c.execute(update_row_sql, (od_alpha, od_beta, od_zeta, id))

    conn.commit();

calculate_bls_occupation_scores()

  0%|          | 0/6441 [00:00<?, ?it/s]

In [74]:
import pandas as pd

def update_single_row(id: str, alpha: float, beta: float, zeta: float):
    update_summary_sql = """
UPDATE bls_matrix
SET
    alpha = :alpha,
    beta = :beta,
    zeta = :zeta
WHERE
    id = :id
"""
    # update the row with the weighted average
    c.execute(update_summary_sql, {"alpha": alpha, "beta": beta, "zeta": zeta, "id": id})    

def sumprod(line_items: list):
    # stuff the rows into a pandas dataframe
    df = pd.DataFrame(line_items, columns=["id", "employment_2022", "alpha", "beta", "zeta"])

    # calculate the weighted average of the scores
    weighted_alpha = (df["employment_2022"] * df["alpha"]).sum() / df["employment_2022"].sum()
    weighted_beta = (df["employment_2022"] * df["beta"]).sum() / df["employment_2022"].sum()
    weighted_zeta = (df["employment_2022"] * df["zeta"]).sum() / df["employment_2022"].sum()

    return weighted_alpha, weighted_beta, weighted_zeta


def calculate_summary_scores_by_industry(industry_code: str):
    # Fetch the occupation codes to summarize.
    occupation_codes = """
SELECT
    id,
    occupation_code
FROM
    bls_matrix
WHERE
    industry_code = :industry_code AND
    occupation_type = 'Summary' AND
    occupation_code != '00-0000'
"""
    occupation_codes = c.execute(occupation_codes, {"industry_code": industry_code}).fetchall()

    # For each row, summarize the Line items.
    for row in occupation_codes:
        id, occupation_code = row

        regexp = occupation_code.rstrip("0").ljust(7, "_")

        # Fetch the set of rows to summarize by looking at codes that start with the industry code
        query_sql = """
SELECT
    id,
    employment_2022,
    alpha,
    beta,
    zeta
FROM
    bls_matrix
WHERE
    industry_code = :industry_code AND
    occupation_code LIKE :regexp    
"""
        line_items = c.execute(query_sql, {"industry_code": industry_code, "regexp": regexp}).fetchall()
        if len(line_items) == 0:
            print(f"No line items found.")
            continue

        weighted_alpha, weighted_beta, weighted_zeta = sumprod(line_items)

        # update the row with the weighted average
        update_single_row(id, weighted_alpha, weighted_beta, weighted_zeta)

    conn.commit()

# Summarize over all industries
def calculate_summary_scores():
    industry_codes_sql = """
SELECT
  DISTINCT industry_code
FROM
  bls_matrix
"""
    industry_codes = c.execute(industry_codes_sql).fetchall()
    for row in tqdm(industry_codes):
        industry_code = row[0]
        calculate_summary_scores_by_industry(industry_code)
        print(f"Finished {industry_code}")

# calculate_summary_scores()

In [75]:
# Update the summaries for the "All industries" rows
def calculate_summary_scores_for_all_industries():
    industry_codes_sql = """
SELECT
    id,
    industry_code
FROM
    bls_matrix
WHERE
    occupation_type = 'Summary' AND
    occupation_code = '00-0000' AND
    alpha IS NULL
"""
    for row in tqdm(c.execute(industry_codes_sql).fetchall()):
        id, industry_code = row

        query_sql = """
SELECT
    id,
    employment_2022,
    alpha,
    beta,
    zeta
FROM
    bls_matrix
WHERE
    industry_code = :industry_code AND
    occupation_type = 'Line item'
"""
        line_items = c.execute(query_sql, {"industry_code": industry_code}).fetchall()
        if len(line_items) == 0:
            print(f"No line items found.")
            continue

        weighted_alpha, weighted_beta, weighted_zeta = sumprod(line_items)

        # update the row with the weighted average
        update_single_row(id, weighted_alpha, weighted_beta, weighted_zeta)
        print(f"Finished {industry_code}")
    conn.commit()

calculate_summary_scores_for_all_industries()

  0%|          | 0/426 [00:00<?, ?it/s]

Finished TE1000
Finished TE1100
Finished TE1200
Finished 110000
Finished 111000
Finished 112000
Finished 113000
Finished 1131-2
Finished 113300
Finished 114000
Finished 115000
Finished 210000
Finished 211000
Finished 212100
Finished 212200
Finished 212300
Finished 213000
Finished 220000
Finished 221000
Finished 221100
Finished 221110
Finished 221111
Finished 221112
Finished 221113
Finished 221114
Finished 221115
Finished 221116
Finished 221117
Finished 221118
Finished 221200
Finished 221300
Finished 230000
Finished 236000
Finished 236100
Finished 236200
Finished 237000
Finished 237100
Finished 237130
Finished 237200
Finished 237300
Finished 237900
Finished 238000
Finished 238100
Finished 238110
Finished 238140
Finished 238160
Finished 238200
Finished 238210
Finished 238220
Finished 238290
Finished 238300
Finished 238310
Finished 238320
Finished 238900
Finished 31-330
Finished 311000
Finished 311100
Finished 311200
Finished 311300
Finished 311400
Finished 311500
Finished 311600
Finished